In [1]:
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from collections import defaultdict  
import os, math, warnings, math, pickle
from tqdm import tqdm
import faiss
import collections
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss
warnings.filterwarnings('ignore')

In [2]:
data_path = 'D:/Desktop/competition/news_article_rs/data/' # 天池平台路径
save_path = 'D:/Desktop/competition/news_article_rs/res/'  # 天池平台路径
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [3]:
os.listdir(data_path)

['articles.csv',
 'articles_emb.csv',
 'DataA1121.md',
 'sample_submit.csv',
 'testA_click_log.csv',
 'testB_click_log_Test_B.csv',
 'train_click_log.csv']

In [4]:
def get_all_click_df(data_path, offline=True):
    if offline:
        all_click = pd.read_csv(data_path + 'train_click_log.csv')
    else:
        trn_click = pd.read_csv(data_path + 'train_click_log.csv')
        tstA_click = pd.read_csv(data_path + 'testA_click_log.csv')
        tstB_click = pd.read_csv(data_path + 'testB_click_log_Test_B.csv')

        all_click = trn_click.append(tstA_click).append(tstB_click)
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

In [5]:
# 读取文章的基本属性
def get_item_info_df(data_path):
    item_info_df = pd.read_csv(data_path + 'articles.csv')
    
    # 为了方便与训练集中的click_article_id拼接，需要把article_id修改成click_article_id
    item_info_df = item_info_df.rename(columns={'article_id': 'click_article_id'})
    
    return item_info_df

In [6]:
# 读取文章的Embedding数据
def get_item_emb_dict(data_path):
    item_emb_df = pd.read_csv(data_path + 'articles_emb.csv')
    
    item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
    item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols])
    # 进行归一化
    item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)

    item_emb_dict = dict(zip(item_emb_df['article_id'], item_emb_np))
    pickle.dump(item_emb_dict, open(save_path + 'item_content_emb.pkl', 'wb'))
    
    return item_emb_dict

In [7]:
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))

In [8]:
# 采样数据
# all_click_df = get_all_click_sample(data_path)

# 全量训练集
all_click_df = get_all_click_df(data_path, offline=False)

# 对时间戳进行归一化,用于在关联规则的时候计算权重
all_click_df['click_timestamp'] = all_click_df[['click_timestamp']].apply(max_min_scaler)

In [9]:
item_info_df = get_item_info_df(data_path)
item_emb_dict = get_item_emb_dict(data_path)

## 获取用户-文章-时间函数

In [10]:
def get_user_item_time(click_df):
    
    click_df = click_df.sort_values('click_timestamp')
    
    def make_item_time_pair(df):
        return list(zip(df['click_article_id'], df['click_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'].apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict

## 获取文章-用户-时间函数

In [11]:
# 根据时间获取商品被点击的用户序列  {item1: [(user1, time1), (user2, time2)...]...}
# 这里的时间是用户点击当前商品的时间，好像没有直接的关系。
def get_item_user_time_dict(click_df):
    def make_user_time_pair(df):
        return list(zip(df['user_id'], df['click_timestamp']))
    
    click_df = click_df.sort_values('click_timestamp')
    item_user_time_df = click_df.groupby('click_article_id')['user_id', 'click_timestamp'].apply(lambda x: make_user_time_pair(x)).reset_index().rename(columns={0: 'user_time_list'})
    
    item_user_time_dict = dict(zip(item_user_time_df['click_article_id'], item_user_time_df['user_time_list']))
    return item_user_time_dict

## 获取历史和最后一次点击

In [12]:
# 获取当前数据的历史点击和最后一次点击
def get_hist_and_last_click(all_click):
    
    all_click = all_click.sort_values(by=['user_id', 'click_timestamp'])
    click_last_df = all_click.groupby('user_id').tail(1)

    # 如果用户只有一个点击，hist为空了，会导致训练的时候这个用户不可见，此时默认泄露一下
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]

    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)

    return click_hist_df, click_last_df

## 获取文章属性特征

In [13]:
# 获取文章id对应的基本属性，保存成字典的形式，方便后面召回阶段，冷启动阶段直接使用
def get_item_info_dict(item_info_df):
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    item_info_df['created_at_ts'] = item_info_df[['created_at_ts']].apply(max_min_scaler)
    
    item_type_dict = dict(zip(item_info_df['click_article_id'], item_info_df['category_id']))
    item_words_dict = dict(zip(item_info_df['click_article_id'], item_info_df['words_count']))
    item_created_time_dict = dict(zip(item_info_df['click_article_id'], item_info_df['created_at_ts']))
    
    return item_type_dict, item_words_dict, item_created_time_dict

## 获取用户历史点击的文章信息

In [14]:
def get_user_hist_item_info_dict(all_click):
    
    # 获取user_id对应的用户历史点击文章类型的集合字典
    user_hist_item_typs = all_click.groupby('user_id')['category_id'].agg(set).reset_index()
    user_hist_item_typs_dict = dict(zip(user_hist_item_typs['user_id'], user_hist_item_typs['category_id']))
    
    # 获取user_id对应的用户点击文章的集合
    user_hist_item_ids_dict = all_click.groupby('user_id')['click_article_id'].agg(set).reset_index()
    user_hist_item_ids_dict = dict(zip(user_hist_item_ids_dict['user_id'], user_hist_item_ids_dict['click_article_id']))
    
    # 获取user_id对应的用户历史点击的文章的平均字数字典
    user_hist_item_words = all_click.groupby('user_id')['words_count'].agg('mean').reset_index()
    user_hist_item_words_dict = dict(zip(user_hist_item_words['user_id'], user_hist_item_words['words_count']))
    
    # 获取user_id对应的用户最后一次点击的文章的创建时间
    all_click_ = all_click.sort_values('click_timestamp')
    user_last_item_created_time = all_click_.groupby('user_id')['created_at_ts'].apply(lambda x: x.iloc[-1]).reset_index()
    
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    user_last_item_created_time['created_at_ts'] = user_last_item_created_time[['created_at_ts']].apply(max_min_scaler)
    
    user_last_item_created_time_dict = dict(zip(user_last_item_created_time['user_id'], \
                                                user_last_item_created_time['created_at_ts']))
    
    return user_hist_item_typs_dict, user_hist_item_ids_dict, user_hist_item_words_dict, user_last_item_created_time_dict

## 获取点击次数最多的topk个文章

In [15]:
# 获取近期点击最多的文章
def get_item_topk_click(click_df, k):
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click

## 召回效果评估函数

In [16]:
# 依次评估召回的前10, 20, 30, 40, 50个文章中的击中率
def metrics_recall(user_recall_items_dict, trn_last_click_df, topk=5):
    last_click_item_dict = dict(zip(trn_last_click_df['user_id'], trn_last_click_df['click_article_id']))
    user_num = len(user_recall_items_dict)
    
    for k in range(10, topk+1, 10):
        hit_num = 0
        for user, item_list in user_recall_items_dict.items():
            # 获取前k个召回的结果
            tmp_recall_items = [x[0] for x in user_recall_items_dict[user][:k]]
            if last_click_item_dict[user] in set(tmp_recall_items):
                hit_num += 1
        
        hit_rate = round(hit_num * 1.0 / user_num, 5)
        print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', user_num)

## 计算相似性矩阵

In [17]:
# 获取文章的属性信息，保存成字典的形式方便查询
item_type_dict, item_words_dict, item_created_time_dict = get_item_info_dict(item_info_df)

### itemcf i2i_sim

- 用户点击的时间权重
- 用户点击的顺序权重
- 文章创建的时间权重

In [18]:
def itemcf_sim(df, item_created_time_dict):
    """
        文章与文章之间的相似性矩阵计算
        :param df: 数据表
        :item_created_time_dict:  文章创建时间的字典
        return : 文章与文章的相似性矩阵
        
        思路: 基于物品的协同过滤(详细请参考上一期推荐系统基础的组队学习) + 关联规则
    """
    user_item_time_dict = get_user_item_time(df)
    
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        # 在基于商品的协同过滤优化的时候可以考虑时间因素
        for loc1, (i, i_click_time) in enumerate(item_time_list):
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for loc2, (j, j_click_time) in enumerate(item_time_list):
                if(i == j):
                    continue
                    
                # 考虑文章的正向顺序点击和反向顺序点击    
                loc_alpha = 1.0 if loc2 > loc1 else 0.7
                # 位置信息权重，其中的参数可以调节
                loc_weight = loc_alpha * (0.9 ** (np.abs(loc2 - loc1) - 1))
                # 点击时间权重，其中的参数可以调节
                click_time_weight = np.exp(0.7 ** np.abs(i_click_time - j_click_time))
                # 两篇文章创建时间的权重，其中的参数可以调节
                created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
                i2i_sim[i].setdefault(j, 0)
                # 考虑多种因素的权重计算最终的文章之间的相似度
                i2i_sim[i][j] += loc_weight * click_time_weight * created_time_weight / math.log(len(item_time_list) + 1)
                
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim_, open(save_path + 'itemcf_i2i_sim.pkl', 'wb'))
    
    return i2i_sim_

In [23]:
i2i_sim = itemcf_sim(all_click_df, item_created_time_dict)

100%|█████████████████████████████████████████████████████████████████████████| 300000/300000 [07:40<00:00, 652.07it/s]


## 基于itemcf召回

In [19]:
# 基于商品的召回i2i
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click, item_created_time_dict, emb_i2i_sim):
    """
        基于文章协同过滤的召回
        :param user_id: 用户id
        :param user_item_time_dict: 字典, 根据点击时间获取用户的点击文章序列   {user1: [(item1, time1), (item2, time2)..]...}
        :param i2i_sim: 字典，文章相似性矩阵
        :param sim_item_topk: 整数， 选择与当前文章最相似的前k篇文章
        :param recall_item_num: 整数， 最后的召回文章数量
        :param item_topk_click: 列表，点击次数最多的文章列表，用户召回补全
        :param emb_i2i_sim: 字典基于内容embedding算的文章相似矩阵
        
        return: 召回的文章列表 [(item1, score1), (item2, score2)...]
    """
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id]
    user_hist_items_ = {user_id for user_id, _ in user_hist_items}
    
    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items):
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items_:
                continue
            
            # 文章创建时间差权重
            created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
            # 相似文章和历史点击文章序列中历史文章所在的位置权重
            loc_weight = (0.9 ** (len(user_hist_items) - loc))
            
            content_weight = 1.0
#             if emb_i2i_sim.get(i, {}).get(j, None) is not None:
#                 content_weight += emb_i2i_sim[i][j]
#             if emb_i2i_sim.get(j, {}).get(i, None) is not None:
#                 content_weight += emb_i2i_sim[j][i]
                
            item_rank.setdefault(j, 0)
            item_rank[j] += created_time_weight * loc_weight * content_weight * wij
    
    # 不足10个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 随便给个负数就行
            if len(item_rank) == recall_item_num:
                break
    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
        
    return item_rank

In [26]:
# 先进行itemcf召回, 为了召回评估，所以提取最后一次点击
metric_recall = False
if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
else:
    trn_hist_click_df = all_click_df

user_recall_items_dict = collections.defaultdict(dict)
user_item_time_dict = get_user_item_time(trn_hist_click_df)

i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))

sim_item_topk = 15
recall_item_num = 40
item_topk_click = get_item_topk_click(trn_hist_click_df, k=40)

for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, \
                                                        i2i_sim, sim_item_topk, recall_item_num, \
                                                        item_topk_click, item_created_time_dict, None)

pickle.dump(user_recall_items_dict, open(save_path + 'itemcf_recall_dict.pkl', 'wb'))

100%|████████████████████████████████████████████████████████████████████████| 300000/300000 [1:38:12<00:00, 50.91it/s]


In [27]:
if metric_recall:
    # 召回效果评估
    metrics_recall(user_recall_items_dict, trn_last_click_df, topk=recall_item_num)

# 构建特征

In [20]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import gc, os
import logging
import time
import lightgbm as lgb
from gensim.models import Word2Vec
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [21]:
data_path = 'D:/Desktop/competition/news_article_rs/data/' # 天池平台路径
save_path = 'D:/Desktop/competition/news_article_rs/res/'  # 天池平台路径
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [22]:
# 节省内存的一个函数
# 减少内存
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df

## 获取历史点击和最后一次点击

In [23]:
# 获取当前数据的历史点击和最后一次点击
def get_hist_and_last_click(all_click):
    all_click = all_click.sort_values(by=['user_id', 'click_timestamp'])
    click_last_df = all_click.groupby('user_id').tail(1)

    # 如果用户只有一个点击，hist为空了，会导致训练的时候这个用户不可见，此时默认泄露一下
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]

    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)

    return click_hist_df, click_last_df

## 读取训练、验证和测试集

In [24]:
def get_trn_val_tst_data(data_path, offline=True):
    if offline:
        click_trn_data = pd.read_csv(data_path+'train_click_log.csv')  # 训练集用户点击日志
        click_trn_data = reduce_mem(click_trn_data)
        click_trn, click_val, val_ans = trn_val_split(click_trn_data, sample_user_nums)
    else:
        click_trn = pd.read_csv(data_path+'train_click_log.csv')
        click_trn = click_trn.append(pd.read_csv(data_path+'testA_click_log.csv'))
        click_trn = reduce_mem(click_trn)
        click_val = None
        val_ans = None
    
    click_tst = pd.read_csv(data_path + 'testB_click_log_Test_B.csv')
    
    return click_trn, click_val, click_tst, val_ans

## 读取Embedding

In [26]:
def trian_item_word2vec(click_df, embed_size=128, save_name='item_w2v_emb.pkl', split_char=' '):
    click_df = click_df.sort_values('click_timestamp')
    # 只有转换成字符串才可以进行训练
    click_df['click_article_id'] = click_df['click_article_id'].astype(str)
    # 转换成句子的形式
    docs = click_df.groupby(['user_id'])['click_article_id'].apply(lambda x: list(x)).reset_index()
    docs = docs['click_article_id'].values.tolist()

    # 为了方便查看训练的进度，这里设定一个log信息
    logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s', level=logging.INFO)

    # 这里的参数对训练得到的向量影响也很大,默认负采样为5
    w2v = Word2Vec(docs, size=embed_size, sg=1, window=15, seed=2020, workers=12, min_count=1, iter=50)
    
    # 保存成字典的形式
    item_w2v_emb_dict = {k: w2v[k] for k in click_df['click_article_id']}
    pickle.dump(item_w2v_emb_dict, open(save_path + 'item_w2v_emb.pkl', 'wb'))
    
    return item_w2v_emb_dict

In [27]:
# 可以通过字典查询对应的item的Embedding
def get_embedding(save_path, all_click_df):
    if os.path.exists(save_path + 'item_content_emb.pkl'):
        item_content_emb_dict = pickle.load(open(save_path + 'item_content_emb.pkl', 'rb'))
    else:
        print('item_content_emb.pkl 文件不存在...')
        
    # w2v Embedding是需要提前训练好的
    if os.path.exists(save_path + 'item_w2v_emb.pkl'):
        item_w2v_emb_dict = pickle.load(open(save_path + 'item_w2v_emb.pkl', 'rb'))
    else:
        item_w2v_emb_dict = trian_item_word2vec(all_click_df)
    
    return item_content_emb_dict, item_w2v_emb_dict

## 读取文章信息

In [28]:
def get_article_info_df():
    article_info_df = pd.read_csv(data_path + 'articles.csv')
    article_info_df = reduce_mem(article_info_df)
    
    return article_info_df

In [29]:
# 获取文章id对应的基本属性，保存成字典的形式，方便后面召回阶段，冷启动阶段直接使用
def get_item_info_dict(item_info_df):
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    item_info_df['created_at_ts'] = item_info_df[['created_at_ts']].apply(max_min_scaler)
    
    item_type_dict = dict(zip(item_info_df['click_article_id'], item_info_df['category_id']))
    item_words_dict = dict(zip(item_info_df['click_article_id'], item_info_df['words_count']))
    item_created_time_dict = dict(zip(item_info_df['click_article_id'], item_info_df['created_at_ts']))
    
    return item_type_dict, item_words_dict, item_created_time_dict

## 读取数据

In [30]:
# 这里offline的online的区别就是验证集是否为空
click_trn, click_val, click_tst, val_ans = get_trn_val_tst_data(data_path, offline=False)

-- Mem. usage decreased to 46.65 Mb (62.5% reduction),time spend:0.00 min


In [31]:
click_tst['user_id'].max()

299999

In [32]:
click_trn_hist, click_trn_last = get_hist_and_last_click(click_trn)

if click_val is not None:
    click_val_hist, click_val_last = click_val, val_ans
else:
    click_val_hist, click_val_last = None, None
    
click_tst_hist = click_tst

In [ ]:
# pos_data_num: 19093 neg_data_num: 9980907 pos/neg: 0.001912952400017353

In [34]:
# 负采样函数，这里可以控制负采样时的比例, 这里给了一个默认的值
def neg_sample_recall_data(recall_items_df, sample_rate=0.001):
    pos_data = recall_items_df[recall_items_df['label'] == 1]
    neg_data = recall_items_df[recall_items_df['label'] == 0]
    
    print('pos_data_num:', len(pos_data), 'neg_data_num:', len(neg_data), 'pos/neg:', len(pos_data)/len(neg_data))
    
    # 分组采样函数
    def neg_sample_func(group_df):
        neg_num = len(group_df)
        sample_num = max(int(neg_num * sample_rate), 1) # 保证最少有一个
        sample_num = min(sample_num, 3) # 保证最多不超过3个，这里可以根据实际情况进行选择
        return group_df.sample(n=sample_num, replace=True)
    
    # 对用户进行负采样，保证所有用户都在采样后的数据中
    neg_data_user_sample = neg_data.groupby('user_id', group_keys=False).apply(neg_sample_func)
    # 对文章进行负采样，保证所有文章都在采样后的数据中
    neg_data_item_sample = neg_data.groupby('sim_item', group_keys=False).apply(neg_sample_func)
    
    # 将上述两种情况下的采样数据合并
    neg_data_new = neg_data_user_sample.append(neg_data_item_sample)
    # 由于上述两个操作是分开的，可能将两个相同的数据给重复选择了，所以需要对合并后的数据进行去重
    neg_data_new = neg_data_new.sort_values(['user_id', 'score']).drop_duplicates(['user_id', 'sim_item'], keep='last')
    
    # 将正样本数据合并
    data_new = pd.concat([pos_data, neg_data_new], ignore_index=True)
    
    return data_new

In [35]:
# 召回数据打标签
def get_rank_label_df(recall_list_df, label_df, is_test=False):
    # 测试集是没有标签了，为了后面代码同一一些，这里直接给一个负数替代
    if is_test:
        recall_list_df['label'] = -1
        return recall_list_df
    
    label_df = label_df.rename(columns={'click_article_id': 'sim_item'})
    recall_list_df_ = recall_list_df.merge(label_df[['user_id', 'sim_item', 'click_timestamp']], \
                                               how='left', on=['user_id', 'sim_item'])
    recall_list_df_['label'] = recall_list_df_['click_timestamp'].apply(lambda x: 0.0 if np.isnan(x) else 1.0)
    del recall_list_df_['click_timestamp']
    
    return recall_list_df_

In [36]:
def get_user_recall_item_label_df(click_trn_hist, click_val_hist, click_tst_hist,click_trn_last, click_val_last, recall_list_df):
    # 获取训练数据的召回列表
    trn_user_items_df = recall_list_df[recall_list_df['user_id'].isin(click_trn_hist['user_id'].unique())]
    # 训练数据打标签
    trn_user_item_label_df = get_rank_label_df(trn_user_items_df, click_trn_last, is_test=False)
    # 训练数据负采样
    trn_user_item_label_df = neg_sample_recall_data(trn_user_item_label_df, sample_rate=0.001)
    
    if click_val is not None:
        val_user_items_df = recall_list_df[recall_list_df['user_id'].isin(click_val_hist['user_id'].unique())]
        val_user_item_label_df = get_rank_label_df(val_user_items_df, click_val_last, is_test=False)
        val_user_item_label_df = neg_sample_recall_data(val_user_item_label_df)
    else:
        val_user_item_label_df = None
        
    # 测试数据不需要进行负采样，直接对所有的召回商品进行打-1标签
    tst_user_items_df = recall_list_df[recall_list_df['user_id'].isin(click_tst_hist['user_id'].unique())]
    tst_user_item_label_df = get_rank_label_df(tst_user_items_df, None, is_test=True)
    
    return trn_user_item_label_df, val_user_item_label_df, tst_user_item_label_df

In [37]:
# 返回多路召回列表或者单路召回
def get_recall_list(save_path, single_recall_model=None, multi_recall=False):
    if multi_recall:
        return pickle.load(open(save_path + 'final_recall_items_dict.pkl', 'rb'))
    
    if single_recall_model == 'i2i_itemcf':
        return pickle.load(open(save_path + 'itemcf_recall_dict.pkl', 'rb'))
    elif single_recall_model == 'i2i_emb_itemcf':
        return pickle.load(open(save_path + 'itemcf_emb_dict.pkl', 'rb'))
    elif single_recall_model == 'user_cf':
        return pickle.load(open(save_path + 'youtubednn_usercf_dict.pkl', 'rb'))
    elif single_recall_model == 'youtubednn':
        return pickle.load(open(save_path + 'youtube_u2i_dict.pkl', 'rb'))

In [38]:
# 将召回列表转换成df的形式
def recall_dict_2_df(recall_list_dict):
    df_row_list = [] # [user, item, score]
    for user, recall_list in tqdm(recall_list_dict.items()):
        for item, score in recall_list:
            df_row_list.append([user, item, score])
    
    col_names = ['user_id', 'sim_item', 'score']
    recall_list_df = pd.DataFrame(df_row_list, columns=col_names)
    
    return recall_list_df

In [39]:
# 读取召回列表
recall_list_dict = get_recall_list(save_path, single_recall_model='i2i_itemcf') # 这里只选择了单路召回的结果，也可以选择多路召回结果
# 将召回数据转换成df
recall_list_df = recall_dict_2_df(recall_list_dict)

100%|███████████████████████████████████████████████████████████████████████| 300000/300000 [00:07<00:00, 41250.96it/s]


In [40]:
# 给训练验证数据打标签，并负采样（这一部分时间比较久）
trn_user_item_label_df, val_user_item_label_df, tst_user_item_label_df = get_user_recall_item_label_df(click_trn_hist, click_val_hist, click_tst_hist, click_trn_last, click_val_last, recall_list_df)

pos_data_num: 19093 neg_data_num: 9980907 pos/neg: 0.001912952400017353


In [46]:
# 将最终的召回的df数据转换成字典的形式做排序特征
def make_tuple_func(group_df):
    row_data = []
    for name, row_df in group_df.iterrows():
        row_data.append((row_df['sim_item'], row_df['score'], row_df['label']))
    
    return row_data

In [47]:
trn_user_item_label_tuples = trn_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
trn_user_item_label_tuples_dict = dict(zip(trn_user_item_label_tuples['user_id'], trn_user_item_label_tuples[0]))

if val_user_item_label_df is not None:
    val_user_item_label_tuples = val_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
    val_user_item_label_tuples_dict = dict(zip(val_user_item_label_tuples['user_id'], val_user_item_label_tuples[0]))
else:
    val_user_item_label_tuples_dict = None
    
tst_user_item_label_tuples = tst_user_item_label_df.groupby('user_id').apply(make_tuple_func).reset_index()
tst_user_item_label_tuples_dict = dict(zip(tst_user_item_label_tuples['user_id'], tst_user_item_label_tuples[0]))

## 制作与用户历史行为相关特征
- 对于每个用户召回的每个商品， 做特征。 具体步骤如下：

    - 对于每个用户， 获取最后点击的N个商品的item_id，
    - 对于该用户的每个召回商品， 计算与上面最后N次点击商品的相似度的和(最大， 最小，均值)， 时间差特征，相似性特征，字数差特征，与该用户的相似性特征

In [48]:
# 下面基于data做历史相关的特征
def create_feature(users_id, recall_list, click_hist_df,  articles_info, articles_emb, user_emb=None, N=1):
    """
    基于用户的历史行为做相关特征
    :param users_id: 用户id
    :param recall_list: 对于每个用户召回的候选文章列表
    :param click_hist_df: 用户的历史点击信息
    :param articles_info: 文章信息
    :param articles_emb: 文章的embedding向量, 这个可以用item_content_emb, item_w2v_emb, item_youtube_emb
    :param user_emb: 用户的embedding向量， 这个是user_youtube_emb, 如果没有也可以不用， 但要注意如果要用的话， articles_emb就要用item_youtube_emb的形式， 这样维度才一样
    :param N: 最近的N次点击  由于testA日志里面很多用户只存在一次历史点击， 所以为了不产生空值，默认是1
    """
    
    # 建立一个二维列表保存结果， 后面要转成DataFrame
    all_user_feas = []
    i = 0
    for user_id in tqdm(users_id):
        # 该用户的最后N次点击
        hist_user_items = click_hist_df[click_hist_df['user_id']==user_id]['click_article_id'][-N:]
        
        # 遍历该用户的召回列表
        for rank, (article_id, score, label) in enumerate(recall_list[user_id]):
            # 该文章建立时间, 字数
            a_create_time = articles_info['created_at_ts'][article_id]
            a_words_count = articles_info['words_count'][article_id]
            single_user_fea = [user_id, article_id]
            # 计算与最后点击的商品的相似度的和， 最大值和最小值， 均值
            sim_fea = []
            time_fea = []
            word_fea = []
            # 遍历用户的最后N次点击文章
            for hist_item in hist_user_items:
                b_create_time = articles_info['created_at_ts'][hist_item]
                b_words_count = articles_info['words_count'][hist_item]
                
                sim_fea.append(np.dot(articles_emb[hist_item], articles_emb[article_id]))
                time_fea.append(abs(a_create_time-b_create_time))
                word_fea.append(abs(a_words_count-b_words_count))
                
            single_user_fea.extend(sim_fea)      # 相似性特征
            single_user_fea.extend(time_fea)    # 时间差特征
            single_user_fea.extend(word_fea)    # 字数差特征
            single_user_fea.extend([max(sim_fea), min(sim_fea), sum(sim_fea), sum(sim_fea) / len(sim_fea)])  # 相似性的统计特征
            
            if user_emb:  # 如果用户向量有的话， 这里计算该召回文章与用户的相似性特征 
                single_user_fea.append(np.dot(user_emb[user_id], articles_emb[article_id]))
                
            single_user_fea.extend([score, rank, label])    
            # 加入到总的表中
            all_user_feas.append(single_user_fea)
    
    # 定义列名
    id_cols = ['user_id', 'click_article_id']
    sim_cols = ['sim' + str(i) for i in range(N)]
    time_cols = ['time_diff' + str(i) for i in range(N)]
    word_cols = ['word_diff' + str(i) for i in range(N)]
    sat_cols = ['sim_max', 'sim_min', 'sim_sum', 'sim_mean']
    user_item_sim_cols = ['user_item_sim'] if user_emb else []
    user_score_rank_label = ['score', 'rank', 'label']
    cols = id_cols + sim_cols + time_cols + word_cols + sat_cols + user_item_sim_cols + user_score_rank_label
            
    # 转成DataFrame
    df = pd.DataFrame( all_user_feas, columns=cols)
    
    return df

In [49]:
# article_info = get_article_info_df()
item_info_df = get_item_info_df(data_path)
item_type_dict, item_words_dict, item_created_time_dict = get_item_info_dict(item_info_df)
article_info = {
    'created_at_ts': item_created_time_dict,
    'words_count': item_words_dict
}
all_click = click_trn.append(click_tst)
item_content_emb_dict, item_w2v_emb_dict = get_embedding(save_path, all_click)

2021-01-14 22:14:43,783:INFO:collecting all words and their counts
2021-01-14 22:14:43,783:INFO:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-01-14 22:14:43,790:INFO:PROGRESS: at sentence #10000, processed 25727 words, keeping 3473 word types
2021-01-14 22:14:43,798:INFO:PROGRESS: at sentence #20000, processed 53883 words, keeping 5811 word types
2021-01-14 22:14:43,807:INFO:PROGRESS: at sentence #30000, processed 84881 words, keeping 7676 word types
2021-01-14 22:14:43,816:INFO:PROGRESS: at sentence #40000, processed 118390 words, keeping 9297 word types
2021-01-14 22:14:43,826:INFO:PROGRESS: at sentence #50000, processed 154179 words, keeping 10844 word types
2021-01-14 22:14:43,836:INFO:PROGRESS: at sentence #60000, processed 192350 words, keeping 12357 word types
2021-01-14 22:14:43,847:INFO:PROGRESS: at sentence #70000, processed 233685 words, keeping 13473 word types
2021-01-14 22:14:43,860:INFO:PROGRESS: at sentence #80000, processed 281335 words, keepin

2021-01-14 22:15:05,210:INFO:worker thread finished; awaiting finish of 8 more threads
2021-01-14 22:15:05,245:INFO:worker thread finished; awaiting finish of 7 more threads
2021-01-14 22:15:05,260:INFO:worker thread finished; awaiting finish of 6 more threads
2021-01-14 22:15:05,277:INFO:worker thread finished; awaiting finish of 5 more threads
2021-01-14 22:15:05,286:INFO:worker thread finished; awaiting finish of 4 more threads
2021-01-14 22:15:05,289:INFO:worker thread finished; awaiting finish of 3 more threads
2021-01-14 22:15:05,316:INFO:worker thread finished; awaiting finish of 2 more threads
2021-01-14 22:15:05,322:INFO:worker thread finished; awaiting finish of 1 more threads
2021-01-14 22:15:05,333:INFO:worker thread finished; awaiting finish of 0 more threads
2021-01-14 22:15:05,334:INFO:EPOCH - 3 : training on 2407290 raw words (2165515 effective words) took 5.1s, 425959 effective words/s
2021-01-14 22:15:06,363:INFO:EPOCH 4 - PROGRESS: at 50.96% examples, 633931 words/s,

2021-01-14 22:15:30,816:INFO:worker thread finished; awaiting finish of 10 more threads
2021-01-14 22:15:30,869:INFO:worker thread finished; awaiting finish of 9 more threads
2021-01-14 22:15:30,896:INFO:worker thread finished; awaiting finish of 8 more threads
2021-01-14 22:15:30,910:INFO:worker thread finished; awaiting finish of 7 more threads
2021-01-14 22:15:30,953:INFO:worker thread finished; awaiting finish of 6 more threads
2021-01-14 22:15:30,971:INFO:EPOCH 8 - PROGRESS: at 99.03% examples, 417312 words/s, in_qsize 5, out_qsize 1
2021-01-14 22:15:30,972:INFO:worker thread finished; awaiting finish of 5 more threads
2021-01-14 22:15:30,983:INFO:worker thread finished; awaiting finish of 4 more threads
2021-01-14 22:15:30,984:INFO:worker thread finished; awaiting finish of 3 more threads
2021-01-14 22:15:30,991:INFO:worker thread finished; awaiting finish of 2 more threads
2021-01-14 22:15:31,005:INFO:worker thread finished; awaiting finish of 1 more threads
2021-01-14 22:15:31,

2021-01-14 22:15:56,302:INFO:worker thread finished; awaiting finish of 11 more threads
2021-01-14 22:15:56,338:INFO:worker thread finished; awaiting finish of 10 more threads
2021-01-14 22:15:56,339:INFO:worker thread finished; awaiting finish of 9 more threads
2021-01-14 22:15:56,416:INFO:worker thread finished; awaiting finish of 8 more threads
2021-01-14 22:15:56,434:INFO:worker thread finished; awaiting finish of 7 more threads
2021-01-14 22:15:56,435:INFO:worker thread finished; awaiting finish of 6 more threads
2021-01-14 22:15:56,478:INFO:worker thread finished; awaiting finish of 5 more threads
2021-01-14 22:15:56,479:INFO:worker thread finished; awaiting finish of 4 more threads
2021-01-14 22:15:56,491:INFO:worker thread finished; awaiting finish of 3 more threads
2021-01-14 22:15:56,498:INFO:worker thread finished; awaiting finish of 2 more threads
2021-01-14 22:15:56,502:INFO:worker thread finished; awaiting finish of 1 more threads
2021-01-14 22:15:56,510:INFO:EPOCH 13 - P

2021-01-14 22:16:19,980:INFO:EPOCH 18 - PROGRESS: at 83.79% examples, 477085 words/s, in_qsize 24, out_qsize 0
2021-01-14 22:16:21,011:INFO:EPOCH 18 - PROGRESS: at 91.47% examples, 437628 words/s, in_qsize 23, out_qsize 0
2021-01-14 22:16:21,741:INFO:worker thread finished; awaiting finish of 11 more threads
2021-01-14 22:16:21,742:INFO:worker thread finished; awaiting finish of 10 more threads
2021-01-14 22:16:21,742:INFO:worker thread finished; awaiting finish of 9 more threads
2021-01-14 22:16:21,786:INFO:worker thread finished; awaiting finish of 8 more threads
2021-01-14 22:16:21,813:INFO:worker thread finished; awaiting finish of 7 more threads
2021-01-14 22:16:21,880:INFO:worker thread finished; awaiting finish of 6 more threads
2021-01-14 22:16:21,894:INFO:worker thread finished; awaiting finish of 5 more threads
2021-01-14 22:16:21,894:INFO:worker thread finished; awaiting finish of 4 more threads
2021-01-14 22:16:21,902:INFO:worker thread finished; awaiting finish of 3 more t

...

2021-01-14 22:17:08,378:INFO:worker thread finished; awaiting finish of 1 more threads
2021-01-14 22:17:08,381:INFO:worker thread finished; awaiting finish of 0 more threads
2021-01-14 22:17:08,381:INFO:EPOCH - 27 : training on 2407290 raw words (2164448 effective words) took 5.2s, 416181 effective words/s
2021-01-14 22:17:09,446:INFO:EPOCH 28 - PROGRESS: at 51.05% examples, 609226 words/s, in_qsize 23, out_qsize 0
2021-01-14 22:17:10,456:INFO:EPOCH 28 - PROGRESS: at 68.76% examples, 517073 words/s, in_qsize 24, out_qsize 0
2021-01-14 22:17:11,492:INFO:EPOCH 28 - PROGRESS: at 82.53% examples, 465364 words/s, in_qsize 23, out_qsize 0
2021-01-14 22:17:12,496:INFO:EPOCH 28 - PROGRESS: at 90.66% examples, 431374 words/s, in_qsize 23, out_qsize 0
2021-01-14 22:17:13,375:INFO:worker thread finished; awaiting finish of 11 more threads
2021-01-14 22:17:13,419:INFO:worker thread finished; awaiting finish of 10 more threads
2021-01-14 22:17:13,462:INFO:worker thread finished; awaiting finish of 

2021-01-14 22:17:34,149:INFO:worker thread finished; awaiting finish of 1 more threads
2021-01-14 22:17:34,153:INFO:worker thread finished; awaiting finish of 0 more threads
2021-01-14 22:17:34,153:INFO:EPOCH - 32 : training on 2407290 raw words (2165323 effective words) took 5.1s, 424177 effective words/s
2021-01-14 22:17:35,187:INFO:EPOCH 33 - PROGRESS: at 51.50% examples, 636814 words/s, in_qsize 23, out_qsize 0
2021-01-14 22:17:36,259:INFO:EPOCH 33 - PROGRESS: at 70.34% examples, 526256 words/s, in_qsize 23, out_qsize 0
2021-01-14 22:17:37,268:INFO:EPOCH 33 - PROGRESS: at 83.28% examples, 473514 words/s, in_qsize 23, out_qsize 0
2021-01-14 22:17:38,276:INFO:EPOCH 33 - PROGRESS: at 91.13% examples, 437160 words/s, in_qsize 23, out_qsize 0
2021-01-14 22:17:39,051:INFO:worker thread finished; awaiting finish of 11 more threads
2021-01-14 22:17:39,102:INFO:worker thread finished; awaiting finish of 10 more threads
2021-01-14 22:17:39,113:INFO:worker thread finished; awaiting finish of 

2021-01-14 22:17:59,968:INFO:worker thread finished; awaiting finish of 5 more threads
2021-01-14 22:17:59,976:INFO:worker thread finished; awaiting finish of 4 more threads
2021-01-14 22:17:59,979:INFO:worker thread finished; awaiting finish of 3 more threads
2021-01-14 22:17:59,980:INFO:worker thread finished; awaiting finish of 2 more threads
2021-01-14 22:17:59,998:INFO:worker thread finished; awaiting finish of 1 more threads
2021-01-14 22:18:00,000:INFO:worker thread finished; awaiting finish of 0 more threads
2021-01-14 22:18:00,001:INFO:EPOCH - 37 : training on 2407290 raw words (2164999 effective words) took 5.1s, 423228 effective words/s
2021-01-14 22:18:01,027:INFO:EPOCH 38 - PROGRESS: at 50.58% examples, 620000 words/s, in_qsize 23, out_qsize 0
2021-01-14 22:18:02,087:INFO:EPOCH 38 - PROGRESS: at 69.12% examples, 517354 words/s, in_qsize 23, out_qsize 0
2021-01-14 22:18:03,159:INFO:EPOCH 38 - PROGRESS: at 82.75% examples, 460587 words/s, in_qsize 23, out_qsize 0
2021-01-14 

2021-01-14 22:18:25,648:INFO:worker thread finished; awaiting finish of 6 more threads
2021-01-14 22:18:25,649:INFO:worker thread finished; awaiting finish of 5 more threads
2021-01-14 22:18:25,703:INFO:worker thread finished; awaiting finish of 4 more threads
2021-01-14 22:18:25,704:INFO:worker thread finished; awaiting finish of 3 more threads
2021-01-14 22:18:25,708:INFO:worker thread finished; awaiting finish of 2 more threads
2021-01-14 22:18:25,713:INFO:worker thread finished; awaiting finish of 1 more threads
2021-01-14 22:18:25,725:INFO:EPOCH 42 - PROGRESS: at 100.00% examples, 425319 words/s, in_qsize 0, out_qsize 1
2021-01-14 22:18:25,725:INFO:worker thread finished; awaiting finish of 0 more threads
2021-01-14 22:18:25,726:INFO:EPOCH - 42 : training on 2407290 raw words (2164976 effective words) took 5.1s, 425212 effective words/s
2021-01-14 22:18:26,742:INFO:EPOCH 43 - PROGRESS: at 50.06% examples, 617699 words/s, in_qsize 23, out_qsize 0
2021-01-14 22:18:27,754:INFO:EPOCH 

2021-01-14 22:18:51,478:INFO:worker thread finished; awaiting finish of 10 more threads
2021-01-14 22:18:51,569:INFO:worker thread finished; awaiting finish of 9 more threads
2021-01-14 22:18:51,570:INFO:worker thread finished; awaiting finish of 8 more threads
2021-01-14 22:18:51,575:INFO:worker thread finished; awaiting finish of 7 more threads
2021-01-14 22:18:51,577:INFO:worker thread finished; awaiting finish of 6 more threads
2021-01-14 22:18:51,608:INFO:worker thread finished; awaiting finish of 5 more threads
2021-01-14 22:18:51,611:INFO:worker thread finished; awaiting finish of 4 more threads
2021-01-14 22:18:51,615:INFO:worker thread finished; awaiting finish of 3 more threads
2021-01-14 22:18:51,634:INFO:EPOCH 47 - PROGRESS: at 99.63% examples, 420873 words/s, in_qsize 2, out_qsize 1
2021-01-14 22:18:51,635:INFO:worker thread finished; awaiting finish of 2 more threads
2021-01-14 22:18:51,645:INFO:worker thread finished; awaiting finish of 1 more threads
2021-01-14 22:18:51

In [51]:
# # 获取训练验证及测试数据中召回列文章相关特征
trn_user_item_feats_df = create_feature(trn_user_item_label_tuples_dict.keys(), trn_user_item_label_tuples_dict, \
                                            click_trn_hist, article_info, item_content_emb_dict)

if val_user_item_label_tuples_dict is not None:
    val_user_item_feats_df = create_feature(val_user_item_label_tuples_dict.keys(), val_user_item_label_tuples_dict, \
                                                click_val_hist, article_info_df, item_content_emb_dict)
else:
    val_user_item_feats_df = None
    
tst_user_item_feats_df = create_feature(tst_user_item_label_tuples_dict.keys(), tst_user_item_label_tuples_dict, \
                                            click_tst_hist, article_info, item_content_emb_dict)

100%|███████████████████████████████████████████████████████████████████████████| 50000/50000 [01:23<00:00, 598.95it/s]


In [52]:
# 保存一份省的每次都要重新跑，每次跑的时间都比较长
trn_user_item_feats_df.to_csv(save_path + 'trn_user_item_feats_df.csv', index=False)

if val_user_item_feats_df is not None:
    val_user_item_feats_df.to_csv(save_path + 'val_user_item_feats_df.csv', index=False)

tst_user_item_feats_df.to_csv(save_path + 'tst_user_item_feats_df.csv', index=False)